# C2 (Client)- run this after running C1(Server)

This scrpit will receive the streaming data and write it into MongoDB.

Before running the following codes, please start MongoDB in the termianl first.



In [1]:
from pymongo import MongoClient
import ast
#The function will send the input list/dictionary to mongoBD server

def sendRecord(document):
    doc = ast.literal_eval(document)       
    
    client = MongoClient()  
    db = client.fit5148_new               
    collection = db.taskc
    
    if type(doc) == dict:
         collection.insert_one(doc)

    client.close()   

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import ast

#reveices the streaming documents from the given port and send this to mongoDB server
def streaming(batch_interval,appName,timeout):
    """
    batch_interval -- the time between each batch
    appName -- the name of the straming application. 
    timeout -- After how long that cancel the execution

    """
    # SparkContext plays an important role in the execution of Spark Applications
    # SparkContext is reaponsible for interacting with programs and spark clusters, including creating RDD
    
    # We add this line to avoid an error : "Cannot run multiple SparkContexts at once".
    # If there is an existing spark context, we will reuse it instead of creating a new context.

    sc = SparkContext.getOrCreate()

    # create a new spark context if there is no existing spark context
    if sc is None:
        # local[*]: run Spark locally with as many working processors as logical cores on your machine.
        sc = SparkContext(master="local[*]", appName=appName)

    # create a streamingcontext based on sparkcontext
    # a batch interval of "batch_interval" seconds 
    ssc = StreamingContext(sc,batch_interval) 
     
    # Create a DStream connecting to hostname:port
    lines = ssc.socketTextStream('localhost', 9999) 
    
    #foreachRDD: push data in each RDD to an external system
    lines.foreachRDD(lambda rdd:  rdd.foreach(sendRecord))

    
    # Start the computation
    ssc.start() 
    
    # Wait for the computation to terminate. 
    # We have added a `timeout` to deliberately cancel the execution after "timeout" seconds. 
    ssc.awaitTermination(timeout=timeout)
    
    #manually stop the streaming context
    ssc.stop()
    sc.stop()
    print('Finish')

In [3]:
streaming(1,'Task3Streaming',1000)

Finish


# Please run following after "Finish" was shown in the previous streaming function
  Use python application to show that streamed data have been added into the database

In [4]:
#Testing
from pprint import pprint
from pymongo import MongoClient
client = MongoClient()  
db = client.fit5148_db 
result = db.taskc
cursor = result.find().count()

In [5]:
cursor

45